plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

Pseudocode:
    1. Import
    2. Load Parameters
    3. For Loop through seizures

In [45]:
def open_csv(mat_file):
    
    with open(mat_file,'r') as sz_mat:
        mat_csv_list = list(csv.reader(sz_mat))

        all_sx = mat_csv_list[0] #collect symptom abbreviations
        sx_col = all_sx.index(sx_input) #find col num of symptom
        csv_rows = np.size(mat_csv_list,0) #find row num 

        for r in range(1,csv_rows):  #collect symptom data for specfic seizure in list
            sx_data.append(mat_csv_list[r][sx_col]) 

        try: 
            mx_index = sx_data.index(mx_input) #find row num of first occurance of mode  
            return mx_index

        except ValueError:
            print('no matching mode')
            

In [46]:
#1. Import
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat

#
import pandas as pd
import datetime
# from plot_change_module import *

In [47]:
#1.5 input symptom, mode, and duration
sx_input = 'lhx'
# sx_input = input('enter 3 digit code for symptom: ')

mx_input = '2'
# mx_input = input('enter mode (1: automatism, 2: tonic, 3: clonic): ')

perdur_input = '10'
# perdur_input = input('enter period of duration: ')

In [48]:
#2. Load Parameters
opscea_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/' # set path

os.chdir(opscea_path)

In [49]:
df = pd.read_excel('OPSCEAparams.xlsx') #dataframe
Ndimensions = df.shape #row, col
params_list = df.values.tolist()

In [50]:
#3. For loop through seizures
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'
os.chdir(avg_change_path)

pt_list = os.listdir(avg_change_path)

new_pt_list = []
anat = []

delete folders that do not contain seizures

In [51]:
for pt_i in range(0,len(pt_list)):
    pt = pt_list[pt_i]
    if pt[0] != '.':
        new_pt_list.append(pt)

In [52]:
for pt_folder in new_pt_list:
    path1 = os.path.join(avg_change_path,pt_folder)
    os.chdir(path1)
    sz_list = os.listdir(path1)

In [53]:
    for sz_i in range(0,len(sz_list)): #collect seizure names  
        sz_name = sz_list[sz_i]
        sx_data = []
        ll_weight_times = []
        

In [54]:
        #ignore folders without seizures
        if sz_name[0] != '.' and sz_name != 'Imaging':  
            sz_name_split = sz_name.split('_') # collect sz_name to list  [EC104, 01]
            sz_suffix = sz_name_split[-1]; # '01'

Extract parameters

In [55]:
            # print(sz_suffix)
            # read_params(Ndimensions,params_list,pt_folder,sz_suffix)
            for num in range(0,Ndimensions[0]): # for all rows in params
                if type(params_list[0][num]) != None:
                    if pt_folder == params_list[0][num]: # if patient exists in path and params
                        if sz_suffix == params_list[1][num]: # if seizure exists in path and params
                            #load data of pt_sz from params main sheet
                            # params_vid_start = 2
                            params_vid_start = params_list[2][num]
                            params_vid_stop = params_list[3][num]

            # for num in range(0,Ndimensions[0]): # for all rows in params 
            #                 if type(params_list[0][num]) != None:
            #                     if pt_folder == params_list[0][num]: # if patient exists in path and params
            #                         if sz_suffix == params_list[1][num]: # if seizure exists in path and params
            #                             #load data of pt_sz from params main sheet
            #                             params_vid_start = params_list[2][num]
            #                             params_vid_stop = params_list[3][num]
            #                             # bl_start = params_list[4][num] 
            #                             # bl_stop = params_list[5][num] 
            #                             # llw = params_list[6][num] 
            #                             # iceeg_scale = params_list[7][num] 
            #                             # fps = params_list[8][num] 
            #                             # cax = params_list[9][num] 
            #                             # gsp = params_list[10][num] 
            #                             # cm = params_list[11][num] 
            #                             # iceegwin = params_list[12][num] 
            #                             # marg = params_list[13][num] 
            #                             # slicebright = params_list[14][num] 

Set path and initialize variables for semiology and time matrix

In [56]:
            path2 = os.path.join(path1,sz_name)
            path3 = os.path.join(path1,'Imaging/elecs')
            

        #load symptom matrix (mat.csv) and time matrix (time_mat.csv) and electrode info over time matrix (sz.mat)
            os.chdir(path2)

            mat_file = '_'.join([sz_name,'mat.csv'])
            time_mat_file = '_'.join([sz_name,'time_mat.csv'])
            sz_mat_file = ''.join([sz_name,'.mat'])

Load semiology matrix          

In [57]:
            ## mat.csv
            mx_index = open_csv(mat_file)

Load time matrix and match up with video and semiology matrix

In [58]:
            ## time_mat.csv
            #MAKE INTO OWN FUNCTION
            #DELETE AND PAD EXCEL SHEETS
            with open(time_mat_file,'r') as sz_time_mat:
                time_mat_csv_list = list(csv.reader(sz_time_mat))
                
                vid_start = time_mat_csv_list[1][0]
                vid_end = time_mat_csv_list[1][1]
                mat_start = time_mat_csv_list[1][2]
                mat_end = time_mat_csv_list[1][3]
                
                td_vid_start = pd.Timedelta(vid_start)
                td_vid_end = pd.Timedelta(vid_end)

                
                td_mat_start = pd.Timedelta(mat_start)
                td_mat_end = pd.Timedelta(mat_end)
                
                td_vid_dur = td_vid_end - td_vid_start
                td_mat_dur = td_mat_end - td_mat_start
                
                #if params vid start == start of video
            # print(params_list[num][:])
#                 td_params_stop = pd.Timedelta(int(params_vid_stop), "s")
                
            
#                 short_vid_start = td_mat_start + td_mat_dur - td_params_stop
                sem_first = td_mat_start - td_vid_start
                sem_last = sem_first + td_mat_dur
                sem_period = [sem_first, sem_last]

Collect linelength weight times

In [59]:
            #COLLECT LL WEIGHT TIMES 
            
            #find first occurance of either 1, 2, or 3 
            td_perdur = pd.Timedelta(int(perdur_input), "s")
            
    
            ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first)
            ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first - td_perdur)
            ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first + td_perdur)

            #collect sorting index
            #collect relevant snippet from ll based on ll_weight_times
            

In [60]:
            ## sz.mat
            # MAKE INTO OWN FUNCTION
            sz_mat = loadmat(sz_mat_file)                            
            
                #load fs, ppEEG, study
            fs = [[element for element in upperElement] for upperElement in sz_mat['fs']]
            ppEEG = [[element for element in upperElement] for upperElement in sz_mat['ppEEG']]
            # study = [[element for element in upperElement] for upperElement in sz_mat['study']]
           
        
            if 'fs' in locals() or 'fs' in globals():
                sfx = fs
                del fs
                
            if 'ppEEG' in locals() or 'ppEEG' in globals():
                d = ppEEG
                del ppEEG

            #load electrodes -- anatomy, eleclabels, elecmatrix
            os.chdir(path3)

In [61]:
            
            elecs_mat = loadmat('clinical_elecs_all.mat')            
        
                #load anatomy
            anatomy = [[element for element in upperElement] for upperElement in elecs_mat['anatomy']]
            # eleclabels = [[element for element in upperElement] for upperElement in elecs_mat['eleclabels']]
            # elecmatrix = [[element for element in upperElement] for upperElement in elecs_mat['elecmatrix']]
            
            
            
            #TRY/EXCEPT INSTEAD
            # make sure variables are present
#             if 'anatomy' in locals() or 'anatomy' in globals():
#                 #collect neuroanatomy by row
#                 for a_row in range(0,np.size(anatomy,0)):
#                     if np.size(anatomy[a_row][3]) == 1:
#                         anat.append(anatomy[a_row][3][0])
#                 del anatomy

                